In [86]:
import numpy as np
import pickle
import time
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
from datetime import datetime

In [94]:
start_run = 199
end_run = 199

output_file_name = 'Run199_list.pkl'
geometry = 'Prototype_Geometry_5-3-24.txt'
boards = 3

In [107]:
Tstamp_us = []
Brd = []
Ch = []
LG = []
HG = []


file_start_time = None

for run_number in range(start_run, end_run+1):
    input_file_name = f"Run{run_number}_list.txt"
    RunTstamp_us = []
    RunBrd = []
    RunCh = []
    RunLG = []
    RunHG = []

    with open(input_file_name) as f:
        lines = f.read().split('\n')

    last_tstamp = 0  # Initialize last timestamp for continuity across files

    for i, line in enumerate(lines):
        if i == 6:  # Line containing the start time
            start_time_str = ' '.join(line.split()[4:-1])  # Extract start time string, excluding "UTC"
            file_start_time = datetime.strptime(start_time_str, "%a %b %d %H:%M:%S %Y")
            file_start_time = file_start_time.timestamp() * 1e6  # Convert to microseconds
            file_start_time = 0
        elif i > 8:
            data = line.split()

            if len(data) == 6:
                RunTstamp_us.append(float(data[0]) + file_start_time)
                last_tstamp = float(data[0]) + file_start_time
                RunBrd.append(int(data[2]))
                RunCh.append(int(data[3]))
                RunLG.append(int(data[4]))
                RunHG.append(int(data[5]))
    
            elif len(data) == 4:
                if last_tstamp is not None:
                    RunTstamp_us.append(last_tstamp)
                RunBrd.append(int(data[0]))
                RunCh.append(int(data[1]))
                RunLG.append(int(data[2]))
                RunHG.append(int(data[3]))
   
    sorted_indices = np.argsort(RunTstamp_us)
    RunTstamp_us = np.array(RunTstamp_us)[sorted_indices]
    RunBrd = np.array(RunBrd)[sorted_indices]
    RunCh = np.array(RunCh)[sorted_indices]
    RunLG = np.array(RunLG)[sorted_indices]
    RunHG = np.array(RunHG)[sorted_indices]
    
    # Create masks for each condition
    mask_1 = (RunBrd == 1)
    mask_2 = (RunBrd == 2)

    # Modify Tstamp_us based on each condition
    #RunTstamp_us[mask_1] -= 20000
    #RunTstamp_us[mask_2] -= 0
    
    Tstamp_us.extend(RunTstamp_us)
    Brd.extend(RunBrd)
    Ch.extend(RunCh)
    LG.extend(RunLG)
    HG.extend(RunHG)
                
                
# Convert timestamps to datetime objects
Tstamp_utc = [datetime.utcfromtimestamp(tstamp_us / 1e6) for tstamp_us in Tstamp_us]


In [104]:
print(Counter(Tstamp_us))

Counter({2589304.456: 64, 2609304.136: 64, 2633312.52: 64, 3344693.368: 64, 3364693.048: 64, 3388701.432: 64, 5736475.968: 64, 5756475.648: 64, 5780484.032: 64, 6954186.072: 64, 6974185.752: 64, 6998194.136: 64, 9684328.784: 64, 9704328.464: 64, 9728336.848: 64, 10248717.048: 64, 10268716.728: 64, 10292725.112: 64, 10856566.616: 64, 10876566.296: 64, 10900574.68: 64, 15077142.232: 64, 15097141.912: 64, 15121150.296: 64, 26593978.08: 64, 26613977.76: 64, 26637986.144: 64, 31279376.408: 64, 31299376.088: 64, 31323384.472: 64, 34097538.456: 64, 34117538.136: 64, 34141546.52: 64, 35986368.72: 64, 36006368.4: 64, 36030376.784: 64, 36447054.56: 64, 36467054.24: 64, 36491062.624: 64, 37209642.616: 64, 37229642.296: 64, 37253650.68: 64, 37488447.752: 64, 37508447.432: 64, 37532455.816: 64, 39733124.896: 64, 39753124.576: 64, 39777132.96: 64, 42003022.384: 64, 42023022.064: 64, 42047030.448: 64, 42816422.336: 64, 42836422.016: 64, 42860430.4: 64, 43031193.792: 64, 43051193.472: 64, 43075201.856

In [108]:
window_size = 100
current_event_id = 1
current_timestamp = Tstamp_us[0]
event_ids = []
for timestamp in Tstamp_us:
    if (abs(timestamp - current_timestamp) <= window_size) | (abs(timestamp - current_timestamp - 20000) <= window_size)| (abs(timestamp - current_timestamp - 44000) <= window_size):
        event_ids.append(current_event_id)
    else:
        current_event_id += 1
        event_ids.append(current_event_id)
        current_timestamp = timestamp
        
event_counts = Counter(event_ids)
mask = [event_counts[event_id] == (boards*64) for event_id in event_ids]

In [109]:
print(Counter(event_ids))

Counter({1: 192, 2: 192, 3: 192, 4: 192, 5: 192, 6: 192, 7: 192, 8: 192, 9: 192, 10: 192, 11: 192, 12: 192, 13: 192, 14: 192, 15: 192, 16: 192, 17: 192, 18: 192, 19: 192, 20: 192, 21: 192, 22: 192, 23: 192, 24: 192, 25: 192, 26: 192, 27: 192, 28: 192, 29: 192, 30: 192, 31: 192, 32: 192, 33: 192, 34: 192, 35: 192, 36: 192, 37: 192, 38: 192, 39: 192, 40: 192, 41: 192, 42: 192, 43: 192, 44: 192, 45: 192, 46: 192, 47: 192, 48: 192, 49: 192, 50: 192, 51: 192, 52: 192, 53: 192, 54: 192, 55: 192, 56: 192, 57: 192, 58: 192, 59: 192, 60: 192, 61: 192, 62: 192, 63: 192, 64: 192, 65: 192, 66: 192, 67: 192, 68: 192, 69: 192, 70: 192, 71: 192, 72: 192, 73: 192, 74: 192, 75: 192, 76: 192, 77: 192, 78: 192, 79: 192, 80: 192, 81: 192, 82: 192, 83: 192, 84: 192, 85: 192, 86: 192, 87: 192, 88: 192, 89: 192, 90: 192, 91: 192, 92: 192, 93: 192, 94: 192, 95: 192, 96: 192, 97: 192, 98: 192, 99: 192, 105: 192, 106: 192, 107: 192, 108: 192, 109: 192, 110: 192, 111: 192, 112: 192, 113: 192, 114: 192, 115: 192,

In [110]:
Brd = [value for i, value in enumerate(Brd) if mask[i]]
Ch = [value for i, value in enumerate(Ch) if mask[i]]
LG = [value for i, value in enumerate(LG) if mask[i]]
HG = [value for i, value in enumerate(HG) if mask[i]]
event_ids = [value for i, value in enumerate(event_ids) if mask[i]]
Tstamp_us = [value for i, value in enumerate(Tstamp_us) if mask[i]]
Tstamp_utc = [value for i, value in enumerate(Tstamp_utc) if mask[i]]

In [111]:
# Define empty lists for x, y, and z positions
x_pos = []
y_pos = []
z_pos = []

# Read coordinates from the text file
with open(geometry, "r") as file:
    coordinates = [eval(line.strip()) for line in file]

# Calculate positions based on Ch and Brd values
for i in range(len(Ch)):
    total_channel = Ch[i] + (Brd[i] * 64)
    if total_channel < len(coordinates):
        x, y, z = coordinates[total_channel]
        x_pos.append(x)
        y_pos.append(y)
        z_pos.append(z)
    else:
        print(f"Error: total_channel value {total_channel} is out of range.")

In [112]:
events = pd.DataFrame({
    'Brd': Brd,
    'Ch': Ch,
    'LG': LG,
    'HG': HG,
    'Tstamp_us': Tstamp_us,
    'Tstamp_utc': Tstamp_utc,
    'event_ids': event_ids,
    'x_pos': x_pos,
    'y_pos': y_pos,
    'z_pos': z_pos,
})

In [113]:
with open(output_file_name, 'wb') as handle:
    pickle.dump(events, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [33]:
print(LG)

[]
